load libraries

In [ ]:


import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from matplotlib import cm
import scipy.integrate as integrate
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import seaborn as sns
import matplotlib.patches as patches
import random

import MOS_design as MD
import mag_design as MGD
import scipy
from scipy import interpolate
# Default value of display.max_rows is 10 i.e. at max 10 rows will be printed.
# Set it None to display all rows in the dataframe
pd.set_option('display.max_rows', None) #To show all values, not just 5
%matplotlib inline
# %matplotlib widget
plt.rcParams['figure.figsize'] = [10, 5]
sns.set(style="darkgrid")


load excel

In [ ]:

# MOSFET data
MOSFETs=pd.read_excel(r'.\Space Components.xls',sheet_name="MOSFETs")
# Se usa del siguiente modo:
#MOSFETs['Id (A @25oC)'][54]=70
pd.options.display.max_columns = None
display(MOSFETs.tail(20))

Input variables and configuration

In [ ]:
Vbus=100 # Tensión en el bus
Vsa_min=80 #Máximo y minimo de la tensión del panel solar
Vsa_max=150
fsw=180e3 #100000 # Frecuencia de conmutación
T=1/fsw
P=500 # Potencia nominal
L_ind=84/1000000

size_D1=50 #Número de elementos en los vectores de D1 y D2.
size_D2=50

# Estos no son límites de operación, que vienen definidos por las variables de arriba. Estos son límites de representación
# de las gráficas como los mapas de calor.
D1_min_sh=0.1
D1_max_sh=1
D2_min_sh=0.0
D2_max_sh=0.7

D1_nom=0.8
D2_nom=0.1
D1_max=0.95
D2_min=0.05


phi_bob=0 #Desfase entre las señales de gobierno D1 y D2.
Lim_sup_D1=0.95 # Límites máximos de operación de los ciclos de trabajo
Lim_inf_D2=0.05

# MOSFET seleccionados en la implementación, según la tabla importada de Excel.
chosen_FET='GS66516B' 
M1_index=MOSFETs[MOSFETs['Part']==chosen_FET].index[0]
M2_index=MOSFETs[MOSFETs['Part']==chosen_FET].index[0]
M3_index=MOSFETs[MOSFETs['Part']==chosen_FET].index[0]
M4_index=MOSFETs[MOSFETs['Part']==chosen_FET].index[0]
M1prot_index=MOSFETs[MOSFETs['Part']=='IXTA26P20P'].index[0]
M3prot_index=MOSFETs[MOSFETs['Part']=='IXTA26P20P'].index[0]
prot_on=1
#Driver
Vdrvr=9
Idrvr=4

Rca_M1=3.2#based on excel file
Rjc_M1=1.5#MOSFETs.loc[M1_index,MOSFETs['Rth jc (oC/W)']]
Rca_M2=3.2
Rjc_M2=1.5#Based on aplication note and datasheet that includes IMS and the rest MOSFETs.loc[M1_index,MOSFETs['Rth jc (oC/W)']]
Rca_M3=3.2
Rjc_M3=1.5#MOSFETs.loc[M1_index,MOSFETs['Rth jc (oC/W)']]
Rca_M4=3.2
Rjc_M4=1.5#MOSFETs.loc[M1_index,MOSFETs['Rth jc (oC/W)']]
Rjc_M1prot=MOSFETs.loc[M1prot_index,'Rth jc (oC/W)']
Rca_M1prot=8.5/2
Rjc_M3prot=MOSFETs.loc[M3prot_index,'Rth jc (oC/W)']
Rca_M3prot=8.5/2

Tjmax_M1=MOSFETs.loc[M1_index,'Tj_max (oC)']
Tjmax_M2=MOSFETs.loc[M2_index,'Tj_max (oC)']
Tjmax_M3=MOSFETs.loc[M3_index,'Tj_max (oC)']
Tjmax_M4=MOSFETs.loc[M4_index,'Tj_max (oC)']
Tjmax_M1prot=MOSFETs.loc[M1prot_index,'Tj_max (oC)']
Tjmax_M3prot=MOSFETs.loc[M3prot_index,'Tj_max (oC)']

Tini=25 #ambient temperature

# Max degradation
aging_n=1 #no aging
aging=1.1 
max_age=1.12
aging_extra=1.2

aging_max=0.12 # According to DUsmez Remaining useful...
aging_steps=6 #number of samples to build the dataset
elements=30 # number of samples 

error_sense=50.0 # en tanto por mil

# magnetic material E30/15/7 3F3
c_ind=6.198/10000000000000
x_ind=1.721
y_ind=2.736
Vol_ind=0.000004
N_ind=40
Ae_ind=0.00006
fw_ind=0.3
WA_ind=0.00009
lm_ind=0.056
rho=1/57000000
Awire_ind=WA_ind*fw_ind/N_ind
Lwire_ind=N_ind*lm_ind

Functions

In [ ]:
##FUNCTIONS MODIFIED
# 

def Gain(D1,D2):
    return D1/(1-D2)

def Vin_f(Vout,D1,D2):
    return Vout*(1-D2)/D1

def D2_f(Vin,Vout,D1):
    return 1-Vin/Vout*D1

def D1_f(Vin,Vout,D2):
    return (1-D2)*Vout/Vin


#
# Este primer período siempre es igual
def Ip1(t,L,Vin,Vout,I1,phi,D1,D2):
    return I1+(Vin-Vout)*t/L

#El segundo período depende de phi+D2 respecto a D1 en su límite de validez, pero no en la ecuación.
def Ip2(t,L,Vin,Vout,I2,phi,D1,D2):
    return np.where(phi<D1,I2+Vin*t/L,I2-Vout*t/L)
    # if (phi<D1):
    #     return I2+Vin*t/L
    # else:
    #     return I2-Vout*t/L

#El tercer período depende de phi+D2 respecto a D1. 
def Ip3(t,L,Vin,Vout,I3,phi,D1,D2):
    return np.where(phi+D2<D1,I3+(Vin-Vout)*t/L,I3+t*0)
    # if phi+D2<D1:
    #     return I3+(Vin-Vout)*t/L
    # else:
    #     return I3+t*0

# Este último período siempre es igual
def Ip4(t,L,Vin,Vout,I4,phi,D1,D2):
    return I4-Vout*t/L



# DEFINICIÓN DE INTERVALOS DE TIEMPO DE CADA PERIODO
# Son funciones auxiliares empleadas por la siguiente función. Generan conjuntos de puntos. Es en general,
#no solo para este caso de 4-switch buuck-boost

def t1(D1,D2,phi,T):
    return np.where(phi<D1,np.linspace(0,T*phi),np.linspace(0,T*D1))

def t2(D1,D2,phi,T):
    return np.where(phi>D1,np.linspace(T*D1,T*phi),np.where(D2+phi>D1,np.linspace(T*phi,D1*T),np.linspace(T*phi,(D2+phi)*T)))


def t3(D1,D2,phi,T):
    return np.where(phi>D1,np.linspace(T*phi,(D2+phi)*T),np.where(D2+phi>D1,np.linspace(D1*T,(D2+phi)*T),np.linspace((D2+phi)*T,D1*T)))

def t4(D1,D2,phi,T):
    return np.where( D2+phi>D1,np.linspace((D2+phi)*T,T),np.linspace(D1*T,T))

    

# Esta función devuelve los intervalos de tiempo t1 a t4 y los valores de comienzo y de final dedichos intervalos.
#Es decir, devuelve 12 elementos, agrupados de 3 en 3. 
# Validada de forma indirecta mediante la ecuación Iind_avg. SE DEBE MEJORAR LA VALIDACIÓN
def intervals(D1,D2,phi,T):
    t1_x=t1(D1,D2,phi,T).T
    df1=pd.DataFrame(t1_x)
    columns=['t1_'+str(i) for i in range(df1.shape[1])]
    df1.columns=columns
    t1_ini=t1_x[:,0]
    df1['t1_ini']=t1_ini
    t1_end = t1_x[:,- 1]
    df1['t1_end']=t1_end

    t2_x=t2(D1,D2,phi,T).T
    t2_ini=t2_x[:,0]
    t2_end = t2_x[:,- 1]
    df2=pd.DataFrame(t2_x)
    columns=['t2_'+str(i) for i in range(df2.shape[1])]
    df2.columns=columns
    df2['t2_ini']=t2_ini
    df2['t2_end']=t2_end

    t3_x=t3(D1,D2,phi,T).T
    t3_ini=t3_x[:,0]
    t3_end = t3_x[:,- 1]
    df3=pd.DataFrame(t3_x)
    columns=['t3_'+str(i) for i in range(df3.shape[1])]
    df3.columns=columns   
    df3['t3_ini']=t3_ini
    df3['t3_end']=t3_end

    t4_x=t4(D1,D2,phi,T).T
    t4_ini=t4_x[:,0]
    t4_end = t4_x[:,- 1]    
    df4=pd.DataFrame(t4_x)

    columns=['t4_'+str(i) for i in range(df4.shape[1])]
    df4.columns=columns
    df4['t4_ini']=t4_ini
    df4['t4_end']=t4_end
    tdf=pd.concat([df1,df2,df3,df4],axis='columns')
    return t1_x,t1_ini,t1_end,t2_x,t2_ini,t2_end,t3_x,t3_ini,t3_end,t4_x,t4_ini,t4_end,tdf
    

# VALORES DE CORRIENTE MEDIA, DE COMIENZO, ETC

# Valor medio de corriente por bobina
# Se debe indicar los valores de corriente al comienzo de cada intervalo y la duración de cada uno de dichos intervalos
# OK Validada en el ejemplo, lo que de forma indirecta valida también la ecuación anterior.
def Iind_avg(I1_ini,I2_ini,I3_ini,I4_ini,T1,T2,T3,T4):
    return 0.5*((I1_ini)*(T4+T1)+I2_ini*(T1+T2)+I3_ini*(T2+T3)+I4_ini*(T3+T4))/(T1+T2+T3+T4)


# Valor final de cada intervalo de corriente por la bobina
# Se debe dar la bobina, tensiones de entrada y salida, el valor de comienzo del periodo I1_ini, ciclos de trabajo,
# desfase y periodo

def currents(L,Vin,Vout,I1_ini,D1,D2,phi,T):
    t=intervals(D1,D2,phi,T)
    Ip4_end=I1_ini
    Ip1_end=Ip1(t[2],L,Vin,Vout,I1_ini,phi,D1,D2)
    Ip2_end=Ip2(t[5]-t[2],L,Vin,Vout,Ip1_end,phi,D1,D2)
    Ip3_end=Ip3(t[8]-t[5],L,Vin,Vout,Ip2_end,phi,D1,D2)
    ipdf=pd.DataFrame({'Ip1_end':Ip1_end,'Ip2_end':Ip2_end,'Ip3_end':Ip3_end,'Ip4_end':Ip4_end})
    return Ip1_end,Ip2_end,Ip3_end,Ip4_end,ipdf

# Corriente inicial del periodo (no de un intervalo):
def Iini(L,Vin,Iin,Vout,D1,D2,phi,T):
    c=currents(L,Vin,Vout,0,D1,D2,phi,T)
    # print(pd.DataFrame(c))
    i=intervals(D1,D2,phi,T)
    A1=np.where(c[3]>0,0.5*(c[0]-0)*(i[2]-i[1])+0.5*np.abs(c[1]+c[0])*(i[5]-i[4])+np.minimum(c[0],c[1])*(i[5]-i[4])+0.5*np.abs(c[2]-c[1])*(i[8]-i[7])+np.minimum(c[2],c[1])*(i[8]-i[7])+0.5*c[2]*(i[11]-i[10]),0.5*(c[0]-0)*(i[2]-i[1])+0.5*np.abs(c[1]+c[0])*(i[5]-i[4])+np.minimum(c[0],c[1])*(i[5]-i[4])+0.5*np.abs(c[2]-c[1])*(i[8]-i[7])+np.minimum(c[2],c[1])*(i[8]-i[7])+0.5*c[2]*(i[11]-i[10]))

    Iini=Iin/D1-A1/T
    return Iini

# Se basa en la anterior, current function, pero sin dar valor inicial
def currents_L(L,Vin,Vout,Iin,D1,D2,phi,T):
    t=intervals(D1,D2,phi,T)
    Ip4_end=Iini(L,Vin,Iin,Vout,D1,D2,phi,T)
    Ip1_end=Ip1(t[2],L,Vin,Vout,Ip4_end,phi,D1,D2)
    Ip2_end=Ip2(t[5]-t[2],L,Vin,Vout,Ip1_end,phi,D1,D2)
    Ip3_end=Ip3(t[8]-t[5],L,Vin,Vout,Ip2_end,phi,D1,D2)
    ipdf=pd.DataFrame({'Ip1_end':Ip1_end,'Ip2_end':Ip2_end,'Ip3_end':Ip3_end,'Ip4_end':Ip4_end})
    return Ip1_end,Ip2_end,Ip3_end,Ip4_end,ipdf

# Valor eficaz y valor medio de un intervalo (no del período completo)
def Irms_sec(fsw,tfin,tini,Ifin,Iini):
    dt=tfin-tini
    dI=(Ifin-Iini)
    return np.sqrt(fsw*(Iini**2*dt+Iini*dI*dt+dI**2*dt/3))

def Iavg_sec(fsw,tfin,tini,Ifin,Iini):
    dt=tfin-tini
    dI=Ifin-Iini
    return 0.5*(Iini+Ifin)*(tfin-tini)*fsw

def Irms_Mi(D1,D2,phi,fsw,tfin1,tfin2,tfin3,tfin4,tini1,Ifin1,Ifin2,Ifin3,Ifin4):
    Irms_M1=np.where(phi+D2>D1,np.sqrt(Irms_sec(fsw,tfin1,tini1,Ifin1,Ifin4)**2+Irms_sec(fsw,tfin2,tfin1,Ifin2,Ifin1)**2),np.sqrt(Irms_sec(fsw,tfin1,tini1,Ifin1,Ifin4)**2+Irms_sec(fsw,tfin2,tfin1,Ifin2,Ifin1)**2+Irms_sec(fsw,tfin3,tfin2,Ifin3,Ifin2)**2))
    Irms_M2=np.where(phi+D2>D1,np.sqrt(Irms_sec(fsw,tfin3,tfin2,Ifin3,Ifin2)**2+Irms_sec(fsw,tfin4,tfin3,Ifin4,Ifin3)**2),np.sqrt(Irms_sec(fsw,tfin4,tfin3,Ifin4,Ifin3)**2))
    Irms_M3=np.where(phi+D2>D1,np.sqrt(Irms_sec(fsw,tfin1,tini1,Ifin1,Ifin4)**2+Irms_sec(fsw,tfin4,tfin3,Ifin4,Ifin3)**2),np.sqrt(Irms_sec(fsw,tfin1,tini1,Ifin1,Ifin4)**2+Irms_sec(fsw,tfin4,tfin3,Ifin4,Ifin3)**2+Irms_sec(fsw,tfin3,tfin2,Ifin3,Ifin2)**2))
    Irms_M4=np.where(phi+D2>D1,np.sqrt(Irms_sec(fsw,tfin2,tfin1,Ifin2,Ifin1)**2+Irms_sec(fsw,tfin3,tfin2,Ifin3,Ifin2)**2),np.sqrt(Irms_sec(fsw,tfin2,tfin1,Ifin2,Ifin1)**2))
        
    return Irms_M1,Irms_M2,Irms_M3,Irms_M4

losses definition

In [ ]:
def R_interpol (M_i,temp):
    R25=float(MOSFETs.loc[M_i,'Rdson (ohms @25oC)'])# If Part is set as index then it could be used: Dataframe.loc[MOSFET_NAME,'Rdson (ohms @25oC)']
    R125=float(MOSFETs.loc[M_i,'Rdson (ohms @125oC)'])#Dataframe.loc[MOSFET_NAME,'Rdson (ohms@125oC)'] it means @ tj max but keep for consistency
    # print(R25,R125)
    temp_value = np.array([25,MOSFETs.loc[M_i,'Tj_max (oC)']])
    R_value = np.array([R25,R125])
    R_interp = scipy.interpolate.interp1d(temp_value, R_value,kind="slinear")
    temp_new = np.linspace(25, 125, 1000)
    return R_interp(temp)

def cond_losses_temp(sw_losses,cond_losses25,M_i,Irms,aging,Rjc,Rca,Tini,Tmax,df):
    cond_losses=cond_losses25*np.ones(df.shape[0])
    losses=sw_losses+cond_losses25
    # print(losses)
    temp=(Tini)*np.ones(df.shape[0])
    temp_end=temp+0.02
    for i in range (df.shape[0]):
        while ((temp_end[i])>temp[i]+0.01 or (temp_end[i])<temp[i]-0.01):
            # print('temp_end-temp',temp_end-temp)
            temp[i]=temp[i]+0.01#incr_temp
            # print('temp_end',temp_end)
            cond_losses[i]=MD.cond_losses(aging*R_interpol(M_i,temp[i]),Irms[i]) 
            # print('cond', cond_losses[i])  
            # print('sw_losses',sw_losses[i])
            # print(cond_losses[i]+sw_losses[i])
            losses[i]=sw_losses[i]+cond_losses[i]
            # print('losses_tot',losses)
            temp_end[i]=Tini+losses[i]*(Rjc+Rca)
            # print ('tini',Tini)
            # print(losses[i]*(Rjc+Rca))
            if (temp_end[i]>Tmax):
                break
        # temp_final[i]=temp_end[i]
    return cond_losses, temp_end#,losses
    
# cond_losses_temp(np.array(([1.0,1.0])),np.array([1.0,1.0]),53,5*np.array([1,1]),1.1,1,1,25,125,np.array([[1,1],[1,1]]))
# cond_losses_temp(np.array(([1.0,1.0])),np.array([1.0,1.0]),53,5*np.array([1,1]),1.1,1,1,25,125,np.array([[1,1],[1,1]]))

losses

In [ ]:
#MOSFET de Buck
def Losses_M1(D1,Vin,Ion,Ioff,fsw,M_i,Vdr,Idr,Irms,aging,Rjc,Rca,Tini,Tmax,df):
#Si no conmuta, sólo hay pérdidas de conducción
    losses_sw1=MD.sw_losses(Vin,Ion,fsw,MOSFETs['Qg (nC)'][M_i]*10**-9,Idr)+MD.sw_losses(Vin,Ioff,fsw,MOSFETs['Qg (nC)'][M_i]*10**-9,Idr)
    losses_sw2=MD.gate_sw_losses(MOSFETs['Qg (nC)'][M_i]*10**-9,Vdr,fsw)
    losses_sw3=MD.Coss_sw_losses(MOSFETs['Coss (pF)'][M_i]*10**-12,Vin,fsw)
    losses_sw=np.where((D1==1) | (D1==0),0,losses_sw1+losses_sw2+losses_sw3) 

    losses_cond25=MD.cond_losses(aging*MOSFETs['Rdson (ohms @25oC)'][M_i],Irms) 
    losses_cond,Temp_M1=cond_losses_temp(losses_sw,losses_cond25,M_i,Irms,aging,Rjc,Rca,Tini,Tmax,df)

    # Temp_M1=cond_losses_temp(losses_sw,losses_cond25,M_i,Irms,aging,Rjc,Rca,Tini,Tmax,df)[1]
    losses_tot=losses_sw+losses_cond
    Temp_M1=np.where((D1!=0),Temp_M1,Tini)
    return np.where(D1==1,losses_tot,np.where(D1==0,0,losses_tot)),Temp_M1

#     if (D1==1):
# # Si conmuta, hay pérdidas de conducción y conmutación. Además, este dispositivo no conmuta con ZVS en encenddo ni ZCS en apagado



# DIODO de Buck        
def Losses_M2(D1,Vin,Ion,Ioff,fsw,M_i,Vdr,Idr,Irms,aging,Rjc,Rca,Tini,Tmax,df):
    losses_sw1=MD.sw_losses(Vin,Ioff,fsw,MOSFETs['Qg (nC)'][M_i]*10**-9,Idr)
    losses_sw2=MD.gate_sw_losses(MOSFETs['Qg (nC)'][M_i]*10**-9,Vdr,fsw)
    losses_sw3=MD.Coss_sw_losses(MOSFETs['Coss (pF)'][M_i]*10**-12,Vin,fsw)
    losses_sw=np.where((D1==1) | (D1==0),0,losses_sw1+losses_sw2+losses_sw3)
    
    losses_cond25=MD.cond_losses(aging*MOSFETs['Rdson (ohms @25oC)'][M_i],Irms)
    losses_cond,Temp_M2=cond_losses_temp(losses_sw,losses_cond25,M_i,Irms,aging,Rjc,Rca,Tini,Tmax,df)
    
    losses_tot=losses_sw+losses_cond
    Temp_M2=np.where((D1==1)|(D1==0),Tini,Temp_M2)

    return np.where((D1==1)|(D1==0),0,losses_tot),Temp_M2
# #Si no conmuta, no hay pérdidas porque está en abierto
# # Si conmuta, hay pérdidas de conducción y conmutación. Pero este dispositivo conmuta con ZVS en encenddo


    
#DIODO de Boost
def Losses_M3(D2,Vout,Ion,Ioff,fsw,M_i,Vdr,Idr,Irms,aging,Rjc,Rca,Tini,Tmax,df):

    losses_sw1=MD.sw_losses(Vout,Ioff,fsw,MOSFETs['Qg (nC)'][M_i]*10**-9,Idr)
    losses_sw2=MD.gate_sw_losses(MOSFETs['Qg (nC)'][M_i]*10**-9,Vdr,fsw)
    losses_sw3=MD.Coss_sw_losses(MOSFETs['Coss (pF)'][M_i]*10**-12,Vout,fsw)
    losses_sw=np.where((D2==1)|(D2==0),0,losses_sw1+losses_sw2+losses_sw3)
    
    losses_cond25=MD.cond_losses(aging*MOSFETs['Rdson (ohms @25oC)'][M_i],Irms)
    losses_cond,Temp_M3=cond_losses_temp(losses_sw,losses_cond25,M_i,Irms,aging,Rjc,Rca,Tini,Tmax,df)

    losses_tot=losses_sw+losses_cond
    Temp_M3=np.where(D2!=1,Temp_M3,Tini)

    return np.where(D2==0,losses_tot,np.where(D2==1,0,losses_tot)),Temp_M3

# # Si conmuta, hay pérdidas de conducción y conmutación. Además, este dispositivo conmuta con ZVS en encenddo

    
#MOSFET de Boost 
def Losses_M4(D2,Vout,Ion,Ioff,fsw,M_i,Vdr,Idr,Irms,aging,Rjc,Rca,Tini,Tmax,df):
    losses_sw1=MD.sw_losses(Vout,Ion,fsw,MOSFETs['Qg (nC)'][M_i]*10**-9,Idr)+MD.sw_losses(Vout,Ioff,fsw,MOSFETs['Qg (nC)'][M_i]*10**-9,Idr)
    losses_sw2=MD.gate_sw_losses(MOSFETs['Qg (nC)'][M_i]*10**-9,Vdr,fsw)
    losses_sw3=MD.Coss_sw_losses(MOSFETs['Coss (pF)'][M_i]*10**-12,Vout,fsw)
    losses_sw=np.where((D2==1) | (D2==0),0,losses_sw1+losses_sw2+losses_sw3)

    losses_cond25=MD.cond_losses(aging*MOSFETs['Rdson (ohms @25oC)'][M_i],Irms)
    losses_cond,Temp_M4=cond_losses_temp(losses_sw,losses_cond25,M_i,Irms,aging,Rjc,Rca,Tini,Tmax,df)
   
    losses_tot=losses_sw+losses_cond  
    Temp_M4=np.where((D2==0) | (D2==1),Tini,Temp_M4)

    return np.where((D2==0) | (D2==1),0,losses_tot),Temp_M4

# # Si conmuta, hay pérdidas de conducción y conmutación. Además, este dispositivo no conmuta con ZVS en encenddo ni ZCS en apagado

    
    
#MOSFET DE PROTECCION AÑADIDO
def Losses_prot(M_i,Irms,aging,Rjc,Rca,Tini,Tmax,df):
    losses_sw=0*np.ones(df.shape[0])
    losses_cond25=MD.cond_losses(aging/2*MOSFETs['Rdson (ohms @25oC)'][M_i],Irms) # divided by 2 due to being two in parallel
    losses_cond,Temp_Mprot=cond_losses_temp(losses_sw,losses_cond25,M_i,Irms,aging,Rjc,Rca,Tini,Tmax,df)   

    losses_tot=losses_cond+losses_sw
    return losses_tot,Temp_Mprot

#HACER PARA MAGNÉTICO
def Losses_ind(c,x,y,Vol,fsw,L,Iac,N,Ae,Irms,Awire,Lwire,rho):
    Bac=L*Iac/2/N/Ae*1000
    losses_sw=MGD.core_losses(c,x,y,Bac,fsw,Vol)
    losses_cond=MGD.copper_losses(Irms,Awire,Lwire,rho)
    return losses_cond+losses_sw




# Valores típicos de c,x,y
(6.198074738669944, 1.7205492791242043, 2.736106449242652)
A=Losses_ind(6.198/10000000000000, 1.721, 2.736,0.000004,180000,84/1000000,6.61,40,0.00006,5.35,0.00009*0.3/40,40*0.056,1/57000000)


heatmap

In [ ]:
D1_r=np.linspace(D1_min_sh,D1_max_sh,size_D1)
D2_r=np.linspace(D2_min_sh,D2_max_sh,size_D2)

CSV = pd.read_csv("Measurements\Bbheatmap3.csv", delimiter=';')
D1_r = np.array(CSV['d1'])
D2_r=np.array(CSV['d2'])
from itertools import product
from typing import ValuesView
#values = np.array(list(product(D1_r,D2_r))) #to get all possible combinations. this if using linspace. if loading from csv use the one bleow
values=np.column_stack((D1_r,D2_r))#I already have all combination, so i only stack both columns

df_hm=pd.DataFrame(values)
df_hm.columns=['D1','D2']
D1_r=df_hm.loc[:,'D1']
D2_r=df_hm.loc[:,'D2']
phi=0
phi_bob=phi*np.ones(df_hm.shape[0])
fsw=180e3
T=1/fsw
P=500
Vbus=100
L_ind=83.5e-6
# df_hm['D1']=D1_r
# df_hm['D2']=D2_r
df_hm['Gain']=D1_r/(1-D2_r)
df_hm['phi']=phi_bob*np.ones(df_hm.shape[0])
df_hm['T']=T*np.ones(df_hm.shape[0])
df_hm['fsw']=fsw*np.ones(df_hm.shape[0])
df_hm['L']=L_ind*np.ones(df_hm.shape[0])
df_hm['Vin']=Vbus*np.ones(df_hm.shape[0])/df_hm.loc[:,'Gain']
df_hm['Vbus']=Vbus*np.ones(df_hm.shape[0])
df_hm['P']=P*np.ones(df_hm.shape[0])
df_hm['Iin(A)']=P/(Vbus/df_hm.loc[:,'Gain'])



c=currents_L(df_hm['L'],df_hm['Vin'],Vbus,df_hm['Iin(A)'],D1_r,D2_r,phi_bob,T)
interv=intervals(D1_r,D2_r,phi_bob,T)
df_hm_los=df_hm.copy()

Irms_M1,Irms_M2,Irms_M3,Irms_M4=Irms_Mi(D1_r,D2_r,phi_bob,fsw,interv[2],interv[5],interv[8],interv[11],interv[1],c[0],c[1],c[2],c[3])
aux_M1,Temp_M1=Losses_M1(D1_r,Vin_f(Vbus,D1_r,D2_r),c[3],c[1],fsw,M1_index,Vdrvr,Idrvr,Irms_M1,aging_n,Rjc_M1,Rca_M1,Tini,Tjmax_M1,df_hm_los)
aux_M2,Temp_M2=Losses_M2(D1_r,Vin_f(Vbus,D1_r,D2_r),c[1],c[3],fsw,M2_index,Vdrvr,Idrvr,Irms_M2,aging_n,Rjc_M2,Rca_M2,Tini,Tjmax_M2,df_hm_los)
aux_M3,Temp_M3=Losses_M3(D2_r,Vbus,c[2],c[0],fsw,M3_index,Vdrvr,Idrvr,Irms_M3,aging_n,Rjc_M3,Rca_M3,Tini,Tjmax_M3,df_hm_los)
aux_M4,Temp_M4=Losses_M4(D2_r,Vbus,c[0],c[2],fsw,M4_index,Vdrvr,Idrvr,Irms_M4,aging_n,Rjc_M4,Rca_M4,Tini,Tjmax_M4,df_hm_los)
aux_protM3,Temp_protM3=Losses_prot(M3prot_index,Irms_M3,aging_n,Rjc_M3prot,Rca_M3prot,Tini,Tjmax_M3prot,df_hm_los)
aux_protM1,Temp_protM1=Losses_prot(M1prot_index,Irms_M1,aging_n,Rjc_M1prot,Rca_M1prot,Tini,Tjmax_M1prot,df_hm_los)

Iac_ind=c[-1].max(axis=1)-c[-1].min(axis=1) #Validated with other implementation. Works
df_hm_los['Iac_ind']=Iac_ind

Iavg_ind=Iind_avg(c[3],c[0],c[1],c[2],interv[2]-interv[1],interv[5]-interv[4],interv[8]-interv[7],interv[11]-interv[10])

Irms1=Irms_sec(1/T,interv[2],interv[1],c[0],c[3])
Irms2=Irms_sec(1/T,interv[5],interv[4],c[1],c[0])
Irms3=Irms_sec(1/T,interv[8],interv[7],c[2],c[1])
Irms4=Irms_sec(1/T,interv[11],interv[10],c[3],c[2])
Irms_ind=np.sqrt(Irms1**2+Irms2**2+Irms3**2+Irms4**2)

df_hm_los['Irms_ind']=Irms_ind
df_hm_los['Ipk_ind']=c[-1].max(axis=1)
df_hm_los['Iavg']=(c[-1].max(axis=1)+c[-1].min(axis=1))/2
aux_L=Losses_ind(c_ind,x_ind,y_ind,Vol_ind,fsw,L_ind,Iac_ind,N_ind,Ae_ind,Irms_ind,Awire_ind,Lwire_ind,rho)

aux=aux_M1+aux_M2+aux_M3+aux_M4+aux_L#+aux_protM1+aux_protM3
losses=aux

df_hm_los['losses M1']=aux_M1
df_hm_los['losses M2']=aux_M2
df_hm_los['losses M3']=aux_M3
df_hm_los['losses M4']=aux_M4
df_hm_los['losses L']=aux_L
df_hm_los['losses M1 prot']=aux_protM1
df_hm_los['Losses M3 prot']=aux_protM3

df_hm_los['Losses']=losses
df_hm_los['Temp_M1']=Temp_M1
df_hm_los['Temp_M2']=Temp_M2
df_hm_los['Temp_M3']=Temp_M3
df_hm_los['Temp_M4']=Temp_M4
df_hm_los['Temp_M1prot']=Temp_protM1
df_hm_los['Temp_M3prot']=Temp_protM3

df_th_meas = pd.DataFrame({
    'D1': D1_r,
    'D2': D2_r,
    'Theoretical losses': losses,

})

dataframe with losses

In [ ]:
df_hm_los

plot heatmap

In [ ]:
heat=df_hm_los.loc[:,['D1','D2','Losses']]
heat=heat.set_index('D2')
heat=heat.pivot(columns='D1')
heat_val=heat.values
D1_x=heat.columns.get_level_values(1)
D2_y=heat.index.values

fig, ax = plt.subplots()
#Plot the surface.
gr=ax.contourf(D1_x, D2_y, heat_val, 50, cmap='hot')#,vmin=6.9,vmax=30)  #'RdYlBu'
cbar=fig.colorbar(gr)
# ticklabs = cbar.ax.get_yticklabels()
# cbar.ax.set_yticklabels(ticklabs, fontsize=20)
# fig.canvas.toolbar_visible = True
# fig.canvas.header_visible = True
# fig.canvas.resizable = True

# Black lines -> max and min gain
#ax.plot(D1_r,D2_f(Vsa_max,Vbus,D1_r),'k')
#ax.plot(D1_r,D2_f(Vsa_min,Vbus,D1_r),'k')

# White lines-> limit the duty cyles due to dead times

#ax.plot(D1_r,D2_f(Vbus,Lim_sup_D1*Vbus,D1_r),'w')
#ax.plot(D1_f(Vbus*(1-Lim_inf_D2),Vbus,D2_r),D2_r,'w')


#ax.plot(D1_r,D2_r,'g')
plt.xlim(0.64,1)
plt.ylim(0,0.4)
# plt.xticks(fontsize = 20)
# plt.yticks(fontsize = 20)
# f = plt.figure()
# f.set_figwidth(20)
# f.set_figheight(10)


plt.show()

MEASUREMENTS

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Read CSV file
CSVM = pd.read_csv("Measurements\Bbheatmap3.csv", delimiter=';')
vout_real = CSV['vout']
iout_real = CSV['iout']
vin_real = CSV['vin']
iin_real = CSV['iin']
d1_real = CSV['d1']
d2_real = CSV['d2']

# Calculate additional parameters
pin_real= vin_real * iin_real
pout_real = iout_real * vout_real
losses_real = pin_real - pout_real
CSV['pin'] = pin_real
CSV['pout'] = pout_real
CSV['losses'] = losses_real
CSV = CSV[['vin', 'iin', 'iout', 'vout', 'd1', 'd2', 'pin', 'pout', 'losses']]
CSV.to_excel("output_data2.xlsx", index=False)
# Reshape losses array
losses_rearr = np.reshape(losses.to_numpy(), (len(np.unique(d1_real)), len(np.unique(d2_real))))
losses_rearr=losses_rearr.T
# Create contour plot
plt.figure()
contour = plt.contourf(np.unique(d1_real), np.unique(d2_real),losses_rearr,70,cmap='hot', linewidth=0)
plt.colorbar()

# Plot additional lines
d1b = np.linspace(np.min(np.unique(d1_real)), np.max(np.unique(d1_real)))
d2b = 1 - (d1b * 80 / 100)
plt.plot(d1b, d2b, 'k', linewidth=2)
d2b = 1 - (d1b * 150 / 100)
plt.plot(d1b, d2b, 'k', linewidth=2)

# Set axis limits
plt.axis([0.64,1,0, 0.4])

# Set colorbar range
#plt.clim(6.9, 30)

# Show the plot
plt.show()

df_th_meas['Meas. losses']=losses


dataframe with difference between theoretical losses and real

In [ ]:
#df_th_meas=df_th_meas.drop(['Loss diff_norm','Meas. losses_norm','Loss diff_norm','Meas. losses2','Loss diff2','Loss diff_norm2'],axis=1)
df_th_meas['Loss diff']=df_th_meas['Meas. losses']-df_th_meas['Theoretical losses']
df_th_meas['Loss diff_norm']=df_th_meas['Meas. losses']/df_th_meas['Meas. losses'].max()
df_th_meas['Theoretical losses_norm']=df_th_meas['Theoretical losses']/df_th_meas['Meas. losses'].max()
df_th_meas['Meas. losses_norm']=df_th_meas['Meas. losses']/df_th_meas['Meas. losses'].max()
df_th_meas['Loss diff_norm']=df_th_meas['Loss diff']/df_th_meas['Meas. losses'].max()
df_th_meas['Meas. losses2']=(df_th_meas['Meas. losses']-df_th_meas['Loss diff'].min())
df_th_meas['Meas. losses2_norm']=(df_th_meas['Meas. losses']-df_th_meas['Loss diff'].min())/df_th_meas['Meas. losses2'].max()
df_th_meas['Loss diff2']=(df_th_meas['Loss diff']-df_th_meas['Loss diff'].min())
df_th_meas['Theoretical losses_norm2']=df_th_meas['Theoretical losses']/df_th_meas['Meas. losses2'].max()
df_th_meas['Loss diff_norm2']=df_th_meas['Loss diff2']/df_th_meas['Meas. losses2'].max()
df_th_meas['Loss diff_norm3']=df_th_meas['Loss diff2']/df_th_meas['Meas. losses2'].max()
df_th_meas['Theoretical losses_norm3']=df_th_meas['Theoretical losses']/df_th_meas['Theoretical losses'].max()
df_th_meas['Meas. losses_norm3']=(df_th_meas['Meas. losses']-df_th_meas['Loss diff'].min())/df_th_meas['Meas. losses2'].max()
print(df_th_meas['Loss diff'].min())
df_th_meas

losses_rearro2 = np.reshape(df_th_meas['Loss diff'].to_numpy(), (len(np.unique(df_th_meas['D1'])), len(np.unique(df_th_meas['D2']))))
losses_rearro2=losses_rearro2.T
contour = plt.contourf(np.unique(df_th_meas['D1']),np.unique(df_th_meas['D2']),losses_rearro2,70,cmap='hot', linewidth=0)
plt.axis([0.64,1,0, 0.4])
plt.colorbar()


In [ ]:
#reploteamos todo pero con la escala de colorbar del measured
#Teorico
losses_rearro = np.reshape(df_th_meas['Theoretical losses'].to_numpy(), (len(np.unique(df_th_meas['D1'])), len(np.unique(df_th_meas['D2']))))
losses_rearro=losses_rearro.T
contour = plt.contourf(np.unique(df_th_meas['D1']),np.unique(df_th_meas['D2']),losses_rearro,70,cmap='hot', linewidth=0)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming you have df_th_meas DataFrame and necessary data

# Create a figure with two subplots side by side
fig, axs = plt.subplots(1, 2, figsize=(12, 5), sharex=True, sharey=True)

# Plot the first contourf in the first subplot
losses_rearro = np.reshape(df_th_meas['Theoretical losses'].to_numpy(), (len(np.unique(df_th_meas['D1'])), len(np.unique(df_th_meas['D2']))))
losses_rearro = losses_rearro.T
contour1 = axs[0].contourf(np.unique(df_th_meas['D1']), np.unique(df_th_meas['D2']), losses_rearro, 70, cmap='hot', linewidth=0)
axs[0].set_title('Theoretical Losses')
axs[0].set_xlabel('D1')
axs[0].set_ylabel('D2')

# Plot the second contourf in the second subplot
losses_rearro2 = np.reshape(df_th_meas['Meas. losses'].to_numpy(), (len(np.unique(df_th_meas['D1'])), len(np.unique(df_th_meas['D2']))))
losses_rearro2 = losses_rearro2.T
contour2 = axs[1].contourf(np.unique(df_th_meas['D1']), np.unique(df_th_meas['D2']), losses_rearro2, 70, cmap='hot', linewidth=0)
axs[1].set_title('Measured Losses')
axs[1].set_xlabel('D1')
axs[1].set_ylabel('D2')

# Determine the common color limits based on the maximum values
max_value = max(losses_rearro.max(), losses_rearro2.max())

# Set color limits for both subplots
contour1.set_clim(0, max_value)
contour2.set_clim(0, max_value)

# Create a single colorbar on the right side
cbar_ax = fig.add_axes([1, 0.15, 0.02, 0.7])  # [x, y, width, height]
cbar = fig.colorbar(contour2, cax=cbar_ax, orientation='vertical')

# Add a title to the colorbar
cbar.set_label('Losses (W)', rotation=270, labelpad=15)

# Adjust layout for better spacing
plt.tight_layout()

# Show the plots
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming you have df_th_meas DataFrame and necessary data

# Create a figure with two subplots side by side
fig, axs = plt.subplots(1, 2, figsize=(12, 5), sharex=True, sharey=True)

# Plot the first contourf in the first subplot
losses_rearro = np.reshape(df_th_meas['Theoretical losses'].to_numpy(), (len(np.unique(df_th_meas['D1'])), len(np.unique(df_th_meas['D2']))))
losses_rearro = losses_rearro.T
contour1 = axs[0].contourf(np.unique(df_th_meas['D1']), np.unique(df_th_meas['D2']), losses_rearro, 70, cmap='hot', linewidth=0)
axs[0].set_title('Theoretical Losses')
axs[0].set_xlabel('D1')
axs[0].set_ylabel('D2')

# Plot the second contourf in the second subplot
losses_rearro2 = np.reshape(df_th_meas['Meas. losses2'].to_numpy(), (len(np.unique(df_th_meas['D1'])), len(np.unique(df_th_meas['D2']))))
losses_rearro2 = losses_rearro2.T
contour2 = axs[1].contourf(np.unique(df_th_meas['D1']), np.unique(df_th_meas['D2']), losses_rearro2, 70, cmap='hot', linewidth=0)
axs[1].set_title('Measured Losses')
axs[1].set_xlabel('D1')
axs[1].set_ylabel('D2')

# Determine the common color limits based on the maximum values
max_value = max(losses_rearro.max(), losses_rearro2.max())

# Set color limits for both subplots
contour1.set_clim(0, max_value)
contour2.set_clim(0, max_value)

# Create a single colorbar on the right side
cbar_ax = fig.add_axes([1, 0.15, 0.02, 0.7])  # [x, y, width, height]
cbar = fig.colorbar(contour2, cax=cbar_ax, orientation='vertical')

# Add a title to the colorbar
cbar.set_label('Losses (W)', rotation=270, labelpad=15)

# Adjust layout for better spacing
plt.tight_layout()

# Show the plots
plt.show()